<a href="https://colab.research.google.com/github/twelvesense/first-repository/blob/master/ExplNode05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로젝트: 가위바위보 분류기를 만들자

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sns

In [1]:
# !mkdir -p /content/data/rock
# !mkdir -p /content/data/paper
# !mkdir -p /content/data/scissor
# !mkdir -p /content/data/test/rock
# !mkdir -p /content/data/test/paper
# !mkdir -p /content/data/test/scissor
!wget -O /content/data/rock/rock.zip https://raw.githubusercontent.com/twelvesense/first-repository/master/data/rock_scissor_paper/rock.zip
!wget -O /content/data/paper/paper.zip https://raw.githubusercontent.com/twelvesense/first-repository/master/data/rock_scissor_paper/paper.zip
!wget -O /content/data/scissor/scissor.zip https://raw.githubusercontent.com/twelvesense/first-repository/master/data/rock_scissor_paper/scissor.zip
!wget -O /content/data/test/rock/rock.zip https://raw.githubusercontent.com/twelvesense/first-repository/master/data/rock_scissor_paper/test_rock.zip
!wget -O /content/data/test/paper/paper.zip https://raw.githubusercontent.com/twelvesense/first-repository/master/data/rock_scissor_paper/test_paper.zip
!wget -O /content/data/test/scissor/scissor.zip https://raw.githubusercontent.com/twelvesense/first-repository/master/data/rock_scissor_paper/test_scissor.zip
!unzip /content/data/paper/paper.zip -d /content/data/paper
!unzip /content/data/rock/rock.zip -d /content/data/rock
!unzip /content/data/scissor/scissor.zip -d /content/data/scissor
!unzip /content/data/test/paper/paper.zip -d /content/data/test/paper
!unzip /content/data/test/rock/rock.zip -d /content/data/test/rock
!unzip /content/data/test/scissor/scissor.zip -d /content/data/test/scissor

--2022-05-17 14:31:55--  https://raw.githubusercontent.com/twelvesense/first-repository/master/data/rock_scissor_paper/rock.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1091596 (1.0M) [application/zip]
Saving to: ‘/content/data/rock/roock.zip’

/content/data/rock/ 100%[===================>]   1.04M  --.-KB/s    in 0.05s   

2022-05-17 14:31:55 (19.7 MB/s) - ‘/content/data/rock/roock.zip’ saved [1091596/1091596]

--2022-05-17 14:31:55--  https://raw.githubusercontent.com/twelvesense/first-repository/master/data/rock_scissor_paper/paper.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... co

In [ ]:
import tensorflow as tf
import numpy as np

print(tf.__version__)
print(np.__version__)

2.8.0
1.21.6


1) 데이터 준비 

2) 딥러닝 네트워크 설계 

3) 학습

4) 테스트(평가)

5) 개선하기